<a href="https://colab.research.google.com/github/Felipanjos/a3_ia_2022.2/blob/main/Projeto_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### D. Preparação dos dados

In [ ]:
import pandas as pd
import numpy as np
import random
import string
import sklearn
from itertools import groupby

In [ ]:
df = pd.read_csv('imdb_top_1000.csv')
# ajuste pra formato de float e typecast
df['Gross'] = df['Gross'].str.replace(',', '')
df['Gross'] = df['Gross'].astype(float)
df = df[['Series_Title', 'Released_Year', 'Runtime', 'Genre', 'IMDB_Rating', 'Meta_score', 'Director', 'No_of_Votes', 'Gross']] # selecinando 9 colunas

##### a) Selecionando 10% das colunas

No dataset utilizado existem somente 9 colunas, dessa forma:

*    10% de 9 = 0.9
*    Aproximando para 1, fica somente uma coluna a ser selecionada como referência para a modificação dos dados



In [ ]:
qtd_colunas = len(df.columns)
p_colunas = round(qtd_colunas * 10 / 100)
coluna_aleatoria = df.sample(n=p_colunas, axis='columns').keys()[0]
colunas_numericas = df.describe().columns
colunas_categoricas = df.describe(exclude=np.number).columns
coluna_aleatoria # escolhendo uma coluna aleatoriamente

##### a) Selecionando 3% dos dados

In [ ]:
df.shape[0]

In [ ]:
p_3 = int(df.shape[0] * 3 / 100)
p_3 # 3% de 1000 é igual a 30

##### a) Excluindo 3% dos dados aleatoriamente

In [ ]:
reg_nulos = df[df[coluna_aleatoria].isnull()]
qtd_reg_nulos = reg_nulos.shape[0]
reg_excluidos = df.sample(n=p_3)
reg_exc_index = reg_excluidos.index.array
df.loc[reg_exc_index, coluna_aleatoria] = np.nan

##### a) Alterando 3% dos dados aleatoriamente

In [ ]:
def gerar_string_aleatoria():
  letters = string.ascii_lowercase
  return ''.join(random.choice(letters) for i in range(10)) 

def gerar_numero_aleatorio():
  return random.randint(0, 999)

In [ ]:
df_selecao_nao_nulo = df[~df[coluna_aleatoria].isnull()]
reg_alterados = df_selecao_nao_nulo.sample(n=p_3)
reg_alter_index = reg_alterados.index.array

In [ ]:
if coluna_aleatoria in colunas_numericas:
    for index in reg_alter_index:
        df.loc[[index],[coluna_aleatoria]] = gerar_numero_aleatorio()
else:
    for index in reg_alter_index:
        df.loc[[index],[coluna_aleatoria]] = gerar_string_aleatoria()

#### b) Limpeza dos dados

i. Codificação One-Hot

In [ ]:
lista_generos = []

for item in df['Genre'].unique():
    lista_generos.extend(item.split(', '))
lista_generos = np.unique(np.array(lista_generos))
lista_generos # obtendo todas as categorias da coluna 'Genre'

In [ ]:
for col in lista_generos:
    df[col] = 1 if col in str(df['Genre']) else 0
# preenchendo as novas colunas 

In [ ]:
df = df.drop('Genre', axis=1)
df #coluna 'Genre' removida e aplicação da Codificação One-hot

ii. Dados numéricos ausentes - Atribuição da mediana geral

In [ ]:
df.info()
# somente as colunas 'Gross', 'Meta_score' e a escolhida aleatoriamente possuem valores nulos

In [ ]:
df.describe(exclude=np.number)

In [ ]:

colunas_com_null = {
    'Meta_score': {
        'index_nulos': df[df['Meta_score'].isnull()].index.to_list(),
        'mean': round(df.describe()['Meta_score']['mean'], 2)
    }, 
    'Gross': {
        'index_nulos': df[df['Gross'].isnull()].index.to_list(),
        'mean': round(df.describe()['Gross']['mean'], 2)
    }
}

In [ ]:
if (coluna_aleatoria in colunas_numericas):
    colunas_com_null[coluna_aleatoria] = {
        'index_nulos': df[df[coluna_aleatoria].isnull()].index.to_list(),
        'mean': round(df.describe()[coluna_aleatoria]['mean'], 2)
    }
    
for coluna in colunas_com_null:
    for index in colunas_com_null[coluna]['index_nulos']:
        df.loc[[index], [coluna]] = colunas_com_null[coluna]['mean']

In [ ]:
df.info()

A atribuição da mediana geral foi utilizada para lidar com os valores nulos das colunas em que existiam, pelos seguintes motivos:

*   Número relativamente pequeno de linhas (1000), o que ocasionaria numa maior escassez de dados caso as linhas com valores nulos fossem removidas
*   Número pequeno de colunas, logo a remoção completa de cada coluna em que houvesse valor nulo acarretaria na perda desnecessária de informações
*   Facilidade na implementação em comparação com o treinamento de predição e atribuição
*   Coesão com o dataset, visto que logo no primeiro quartil a coluna 'Meta_score' apresenta valor próximo à média

Após implementação da correção, o dataset não conta mais com valores nulos.


In [ ]:
df.isnull().values.any()

iii. Escalonamento de Características: Normalização ou Padronização

In [545]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_scaled = df.copy()

for col in ['IMDB_Rating', 'Meta_score', 'No_of_Votes', 'Gross']:
    df_scaled[col] = scaler.fit_transform(df[[col]])
df_scaled = df_scaled[['IMDB_Rating', 'Meta_score', 'No_of_Votes', 'Gross']]